In [16]:
# Setup
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
import numpy as np
import os
from sklearn.model_selection import train_test_split

In [23]:
# Define the paths to your image files
image_files = [
    'cat.0.jpg', 'cat.1.jpg', 'cat.2.jpg', 'cat.3.jpg',
    'dog.0.jpg', 'dog.1.jpg', 'dog.2.jpg', 'dog.3.jpg'
]

# Define the labels for your images (0 for cat, 1 for dog)
labels = [0, 0, 0, 0, 1, 1, 1, 1]

def load_and_preprocess_image(file_path, image_size=(180, 180)):
    """Loads and preprocesses a single image."""
    img = tf.keras.preprocessing.image.load_img(file_path, target_size=image_size)
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = img_array / 255.0  # Normalize pixel values to [0, 1]
    return img_array

# Load and preprocess all images
images = [load_and_preprocess_image(file_path) for file_path in image_files]
images = np.array(images) # Convert to numpy array

# Convert labels to a numpy array
labels = np.array(labels)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, random_state=123)

# Convert labels to categorical format (one-hot encoding)
y_train = tf.keras.utils.to_categorical(y_train, num_classes=2)
y_val = tf.keras.utils.to_categorical(y_val, num_classes=2)

# Create TensorFlow datasets from the NumPy arrays
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(32)
validation_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val)).batch(32)

In [24]:
# Define a simple CNN model
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(180, 180, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax')  # 2 output classes: cat and dog
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [25]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy', # Use categorical_crossentropy for one-hot encoded labels
              metrics=['accuracy'])

history = model.fit(train_dataset, epochs=5, validation_data=validation_dataset) # Use the tf.data.Dataset


Epoch 1/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 0.5000 - loss: 0.6950 - val_accuracy: 0.0000e+00 - val_loss: 23.4280
Epoch 2/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 849ms/step - accuracy: 0.6667 - loss: 11.5319 - val_accuracy: 0.0000e+00 - val_loss: 6.0797
Epoch 3/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 825ms/step - accuracy: 0.6667 - loss: 2.4005 - val_accuracy: 1.0000 - val_loss: 0.0149
Epoch 4/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 551ms/step - accuracy: 0.3333 - loss: 3.5104 - val_accuracy: 1.0000 - val_loss: 0.0733
Epoch 5/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 495ms/step - accuracy: 0.3333 - loss: 1.9710 - val_accuracy: 1.0000 - val_loss: 0.4696


In [26]:
import numpy as np
from tensorflow.keras.preprocessing import image

# Load your image
img = image.load_img('dog.105.jpg', target_size=(180, 180))
img_array = image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)  # Batch dimension

# Predict
pred = model.predict(img_array)
print("Prediction: ", "Dog" if pred[0][0] > 0.5 else "Cat")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
Prediction:  Dog
